# Dask DataFrame (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "Dask DataFrame"
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

* https://docs.dask.org/en/stable/generated/dask.dataframe.groupby.DataFrameGroupBy.apply.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.Series.map_overlap.html#dask.dataframe.Series.map_overlap
* https://docs.dask.org/en/stable/_modules/dask/dataframe/rolling.html#Rolling.apply
* https://dask-sql.readthedocs.io/en/latest/quickstart.html

## Задачи для совместного разбора

1. Загрузите данные о пользователях из архива каталога users. Посчитайте средний возраст каждого представленного пола.

2. Создайте bag на основе данных из файла users.txt. Преобразуйте его в dataframe и объедините с фреймом из задачи 1. 


3. Вычислите произведение каждой пары последовательно идущих возрастов людей.

4. Используя dask_sql, найдите топ-5 возрастов, имеющих наибольшее кол-во представителей.

5. Создайте функцию rename_gender и сделайте выборку из таблицы с ее использованием при помощи dask_sql

## Лабораторная работа 13

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.DataFrame` и средствами пакета `dask`, если в задании не сказано обратного. Переход от коллекций `dask` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.DataFrame` (то есть то, что было получено до вызова `compute`, а не после).

In [1]:
import dask.dataframe as dd

<p class="task" id="1"></p>

1\. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Для каждого автора рассчитайте максимальное количество ингредиентов, которые используются в его рецептах.

In [294]:
dtype={'minutes': 'float64',
       'n_steps': 'float64',
       'id': 'object'}
recipes = dd.read_csv("./recipes_full/*.csv", dtype=dtype, parse_dates=['submitted'])
recipes

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
npartitions=8,,,,,,,,
,object,object,float64,int64,datetime64[ns],float64,object,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [43]:
%%time
recipes.groupby('contributor_id')['n_ingredients'].max().compute()

Wall time: 7.39 s


contributor_id
27            10
1530          15
1531          13
1533          25
1534          20
              ..
2000111609     9
211616         8
254715         8
438612         7
795406        16
Name: n_ingredients, Length: 27926, dtype: int64

<p class="task" id="2"></p>

2\. Удалите строки, которые содержат пропуске в столбце `contributor_id` и приведите его тип к целочисленному. Для каждого `contributor_id` найдите топ-5 слов, которых этот пользователь наиболее часто использовал в названиях своих рецептов. Для разбиения на слова воспользуйтесь методом `.str.split`.

Выведите на экран статистику для пользователя с ID 1530.

In [277]:
recipes_nonan = recipes.dropna(subset=['contributor_id']).astype({'contributor_id': 'int64'})
recipes2 = recipes_nonan[['contributor_id', 'name']]
recipes2['name_words'] = recipes2['name'].str.split()

rec_exploded = recipes2.explode('name_words')
cnt_name_words = rec_exploded.groupby('contributor_id')['name_words'].value_counts().persist()

KeyboardInterrupt: 

In [ ]:
rec_exploded

In [274]:
test.groupby('contributor_id')['name_words'].value_counts()

contributor_id  name_words 
1533            1983           1
                touffe         1
                vegetable      1
                venus          1
                winterfresh    1
                              ..
2001051242      panne          1
                pickaroon      1
                pliaf          1
                rooty          1
                soho           1
Name: count, Length: 9988, dtype: int64

In [100]:
all_words_count(word_count(recipes2[['name_words']].head()),word_count(recipes2[['name_words']].head())).compute()

AttributeError: 'Index' object has no attribute '_get_attributes_dict'

In [58]:
recipes2[['name_words']].head()

,name_words
0,"[vant, ivoire, mickies, nothing]"
1,"[kremsils, mariposa, baccala, cookies, class, ..."
2,"[tania, lander]"
3,"[heloise, milli, asher, doogh, zojirushi]"
4,"[nutty, chocolate, chunk, cookies]"


<p class="task" id="3"></p>

3\. В архиве `stats.zip` находятся файлы, содержащие статистику по рецептам. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `stats`. Объедините две таблицы `recipes` и `stats`. Для каждого рецепта посчитайте среднее количество лайков за период, начиная с даты публикации рецепта и до 1 апреля 2023 года (включительно). Выведите на экран информацию о рецепте, имеющем максимальное значение этой величины.

In [297]:
!pip install --upgrade dask

  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0
  Attempting uninstall: dask
    Found existing installation: dask 2023.3.2
    Uninstalling dask-2023.3.2:
      Successfully uninstalled dask-2023.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.3.2 requires dask==2023.3.2, but you have dask 2023.5.0 which is incompatible.


In [298]:
full_dd.head()

AttributeError: 'Index' object has no attribute '_get_attributes_dict'

In [295]:
stats = dd.read_csv("./stats/*.csv")
full_dd = recipes.merge(stats, left_on='id', right_on='recipe_id')


<p class="task" id="4"></p>

4\. Используя `dask_sql`, выберите из таблицы `recipes` id, название и описание тех рецептов, которые были добавлены в 2010 году. Выведите полученный результат на экран в виде `pd.DataFrame`. Выведите на экран количество  рецептов, которые были добавлены в 2010 году.

<p class="task" id="5"></p>

5\. Создайте функцию `get_recipe_types`, которая определяет сложность рецепта по следующему правилу: если кол-во ингредиентов не больше 5, то рецепт простой ("easy"); если не кол-во ингредиентов не больше 15, то рецепт средней сложности ("medium"); иначе рецепт является сложным ("hard"). 

```python
def get_type(ingredients: int) -> str:
    pass
```

Зарегистрируйте эту функцию для применения в SQL-запросе.

<p class="task" id="6"></p>

6\. Используя `dask_sql`, посчитайте количество рецептов с группировкой по их типу (простой, средний или сложный). Для определения типа рецепта в SQL-запросе воспользуйтесь функцией, зарегистрированной в предыдущем задании. Выведите на экран в виде серии с индексами "easy", "medium" и "hard". 

<p class="task" id="7"></p>

7\. Используя `dask_sql`, объедините таблицы и посчитайте и выведите на экран, сколько рецептов являются верифицированными и не верифицированными. Если информация о верификации рецепта отсутствует, то считайте его не верифицированным. Вся логика подсчета значений должна быть реализована в виде запроса на языке SQL.

<p class="task" id="8"></p>

8\. В файлах архива `site_logs.zip` находятся логи системы. Считайте эти логи в виде `dask.bag`, выделите их них дату, время, id службы, уровень лога и сообщение. Объедините дату и время в одну строку в формате "ГГГГММДД ЧЧММСС". В итоге каждый элемент `Bag` должен представлять собой словарь с ключами `datetime`, `service_id`, `msg`, `level`.

Преобразуйте `dask.bag` в `dask.dataframe`, предварительно оставив только логи уровня "INFO". Подтвердите корректность решения, выведя на экран уникальные значения столбца `level` полученной таблицы.

```
date(ymd)   time(HMS)   service_id level   msg
081110      103041      34         INFO    dfs.FSNamesystem: BLOCK* NameSystem.delete: ...
```

<p class="task" id="9"></p>

9\. Добавьте в таблицу из предыдущего задания столбец `is_delete_op`, который содержит флаг: является ли данная операция удалением чего-либо или нет. Для выяснения этого факта проанализизируйте сообщение лога. 

Для каждого лога подсчитайте, сколько из записанных перед ним последних 100 логов являются операциями удаления. Для вычислений используйте метод `map_overlap`. Посчитайте и выведите на экран, сколько существует подпоследовательностей длины 100, состоящих не менее чем из 75 сообщений, содержащих операции удаления.

<p class="task" id="10"></p>

10\. Сгруппируйте таблицу с логами по часу, в котором был оставлен лог. Под часом имеется в виду час конкретного дня, то связь связка год-месяц-день-час.  Посчитайте, сколько логов было записано за каждый час. Визуализируйте полученную динамику. Добавьте подписи засечек, подписи осей и название рисунка.